<a href="https://colab.research.google.com/github/JaeHeee/Pytorch_Tutorial/blob/main/code/AUTOGRAD_AUTOMATIC_DIFFERENTIATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AUTOGRAD: 자동 미분

PyTorch의 모든 신경망의 중심에는 `autograd` 패키지가 존재.  
`autograd` 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공. 이는 define-by-run 프레임워크로, 이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라진다.

### Tensor

패키지의 중심에는 `torch.Tensor` 클래스가 존재. 만약 `.requires_grad` 속성을 True로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적하기 시작. 계산이 완료된 후, `.backward()`를 호출하여 모든 gradient를 자동을 계산가능. 이 Tensor의 변화도는 `.grad` 속성에 누적됨

Tensor가 기록을 추적하는 것을 중단하게 하려면, `.detach()`를 호출하여 연산 기록으로부터 분리하여 이후 연산들이 추적되는 것을 방지 가능

기록을 추적하는 것(과 메모리를 사용하는 것)을 방지하기 위해, 코드 블럭을 `with torch.no_grad():` 로 감쌀 수 있다. 이는 특히 gradient는 필요없지만, `requires_grad=True` 가 설정되어 학습 가능한 매개변수를 갖는 모델을 evaluate할 때 유용

Autograd 구현에서 매우 중요한 또 다른 클래스는 `Function` 클래스 Tensor 와 Function 은 서로 연결되어 있으며, 모든 연산 과정을 encode하여 acyclic graph를 생성. 각 tensor `.grad_fn` 속성을 갖고 있는데, 이는 Tensor 를 생성한 Function 을 참조 (단, 사용자가 만든 Tensor는 예외로, 이 때 `grad_fn`은 `None`)

derivatives를 계산하기 위해서는 Tensor의 `.backward()` 를 호출. 만약 Tensor 가 scalar인 경우(예. 하나의 요소 값만 갖는 등)에는 `backward` 에 인자를 정해줄 필요가 없지만 여러 개의 요소를 갖고 있을 때는 tensor의 모양을 `gradient` 의 인자로 지정할 필요가 있다.

In [ ]:
import torch

tensor를 생성하고 `requires_grad=True` 를 설정하여 연산을 기록

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


tensor에 연산을 수행

In [ ]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y는 연산의 결과로 생성된 것이므로 `grad_fn`을 가진다.

In [ ]:
print(y.grad_fn)

y에 다른 연산을 수행

In [ ]:
z = y*y*3 # z = 3(x+2)^2
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


`.requires_grad(...)`는 기존 Tensor의 `requires_grad` 값을 in-place 하여 변경. 입력값이 지정되지 않으면 기본값은 False

In [ ]:
a = torch.randn(2, 2)
a = ((a*3)/(a-1))
print(a.requires_grad)

False


In [ ]:
a.requires_grad_(True)
print(a.requires_grad)

True


In [ ]:
b = (a*a).sum()
print(b.grad_fn)

### Gradient

out 은 하나의 스칼라 값만 갖고 있기 때문에, `out.backward()` 는 out.backward(torch.tensor(1.)) 과 동일

In [ ]:
out

tensor(27., grad_fn=<MeanBackward0>)

In [ ]:
out.backward()

gradients d(out)/dx 를 출력

In [ ]:
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [ ]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


`out`을 Tensor "o"라고 하면, 다음과 같이 계산가능


$o = \frac{1}{4}\Sigma_iz_i$ 이고, $z_i = 3(x_i+2)^2$ 이므로 $z_i|_{x_i=1}=27$. 따라서, $\frac{\partial o}{\partial x_i}=\frac{3}{2}(x_i+2)$이므로 $\frac{\partial o}{\partial x_i}|_{x_i=1}=\frac{9}{2}=4.5$

vector 함수 $\vec{y}=f(\vec{x})$ 에서 $\vec{x}$에 대한 $\vec{y}$의 gradient는 Jacobian Matrix 이다.

$\begin{pmatrix} \frac{\partial y_1}{\partial x_1}\ ... \ \frac{\partial y_1}{\partial x_n}\\... \\ \frac{\partial y_m}{\partial x_1}\ ... \ \frac{\partial y_m}{\partial x_n} \end{pmatrix}$

일반적으로, `torch.autograd` 는 vector-Jacobian product를 계산하는 엔진. 즉, vector $v=(v_1\ v_2\ ... v_m)^T$ 에 대해 $v^T \cdot J$을 연산. 만약 v가 scala 함수 $l=g(\vec{y})$의 gradient인 경우 $v=(\frac{\partial l}{\partial y_1}\ ... \frac{\partial l}{\partial y_m})^T$ 이며, chain rule에 따라 vector-Jacobian product는 $\vec{x}$에 대한 l의 gradient가 된다.

$J^T\cdot v=\begin{pmatrix} \frac{\partial y_1}{\partial x_1}\ ... \ \frac{\partial y_m}{\partial x_1}\\... \\ \frac{\partial y_1}{\partial x_n}\ ... \ \frac{\partial y_m}{\partial x_n} \end{pmatrix}\begin{pmatrix} \frac{\partial l}{\partial y_1}\\ .\\.\\\frac{\partial l}{\partial y_m} \end{pmatrix}=\begin{pmatrix} \frac{\partial l}{\partial x_1}\\ .\\.\\\frac{\partial l}{\partial x_n} \end{pmatrix}$

여기서 $v^T \cdot J$ 은 $J^T\cdot v$를 취했을 때의 열 vector로 취급할 수 있는 행 vector를 가진다.

vector-Jacobian product의 이러한 특성은 scala가 아닌 출력을 갖는 모델에 외부 변화도를 제공(feed)하는 것을 매우 편리하게 해준다.

vector-Jacobian product 예시

In [ ]:
x = torch.randn(3, requires_grad=True)

y = x*2
while y.data.norm() < 1000: # norm -> l2 norm
    y = y * 2

print(y)ㅁ

tensor([ 429.5515,  194.7347, 1144.1230], grad_fn=<MulBackward0>)


이 경우 y는 더 이상 scalar 값이 아님. `torch.autograd`는 전체 Jacobian을 직접 계산할수는 없지만, vector-Jacobian product는 간단히 `backward`에 해당 vector를 인자로 제공

In [ ]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


In [ ]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False


또는 `.detach()`를 호출하여 content는 같지만 `require_grad`가 다른 새로운 Tensor를 가져온다.

In [ ]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)


[autograd.Function 관련문서](https://pytorch.org/docs/stable/autograd.html#function)